![](https://api.brandy.run/core/core-logo-wide)

# Object Oriented Programming:  Practical Example

En esa lección veremos un ejemplo practico del uso de los conceptos de Object Oriented Programming, creando diferentes clases y interactuando entre los objetos para programar un juego simples de [Blackjack](https://en.wikipedia.org/wiki/Blackjack).

In [2]:
# Import libraries
from numpy import random

## Card

Vamos empezar nuestro codigo programando el primer de los elementos, una única carta. Esos objetos deben contener 2 atributos minimamente, el valor y el palo. Según vayamos avanzando y programando otros objetos, tendremos que volver hacia atrás para añadir o cambiar atributos o metodos de esa clase.

In [159]:
class Card:
    def __init__(self, value, suit):
        self.value = value
        self.suit = suit
        vals = {"J":10,"Q":10,"K":10,"A":1}
        self.actual_value = value if isinstance(value,int) else vals.get(value)
        
    def __repr__(self):
        if self.suit in ["♥","♦"]:
            return f"\x1b[31m{str(self.value).rjust(2,' ')}{self.suit}\x1b[0m"
        return f"{str(self.value).rjust(2,' ')}{self.suit}"
    
    def __str__(self):
        return f"┌───┐\n|{self.__repr__()}|\n└───┘"
    
    def __add__(self, other):
        return self.actual_value + other
    
    def __radd__(self, other):
        return self.actual_value + other
    
    def __lt__(self, other):
        return self.actual_value < other
    
    def __gt__(self, other):
        return self.actual_value > other
    
    def __eq__(self, other):
        if isinstance(other,int):
            return self.actual_value == other
        elif isinstance(other,Card):
            return self.actual_value == other.actual_value

In [52]:
c = Card("J","♥")

In [53]:
c

 J♥

In [54]:
print(c)

┌───┐
| J♥|
└───┘


In [55]:
c.value, c.suit

('J', '♥')

In [56]:
c.actual_value

10

In [57]:
c_2 = Card(3,"♥")
c_2

 3♥

In [58]:
c + c_2

13

In [59]:
c + 7

17

## Dunder methods
Las clases en python poséen algunos métodos especiales, llamados `dunder` o `magic` methods. La palabra `dunder` es abreviatura de `double underscore`, pues esos métodos se identifican por sus nombres empezaren y terminaren con dos barras bajas (`_`).

Los métodos con ese nombre tienen una característica especial, se relacionan con funciones y operadores `externos` a la clase. El más conocido de eses métodos és el `__init__`, que al contrário de otros métodos, raramente llamamos segun la sintáxis normal, i.e.: `Class.method()`. Salvo en casos en que usamos el `super()` o estamos programando detalles internos de una clase, no llamaremos `Class.__init__()`.

El método constructor se llama cuando sintaticamente usamos una clase como si fuera una función.

```python
object = Class()
```

Python internamente sabe que si estamos llamando a una clase como si fuera a una función, nos estamos referiendo en verdad al método `__init__`, y esa conexión se hace automaticamente. Otros `dunder methods` nos permiten conectar con otras funciones y operadores de python. Por ejemplo:

- `__str__` : Se relaciona con el casting a `str()` y por consecuencia con la función `print`.
- `__len__` : Con la función de mismo nombre, `len()`
- `__gt__`, `__lt__` : Con los operadores `>` y `<` respectivamente.
- [etc...](https://docs.python.org/3/reference/datamodel.html#special-method-names)

## Deck
Si los primeros elementos fueran las cartas, el segundo debe ser la baraja para contenerles. 🃏

Al contrário de las cartas, no necesitamos parámetros para la creación de las barajas, siempre se crearan con todas las 52 cartas. Necesitamos programar un método `shuffle` para barajar y otro `draw` para sacar una siguiente carta.

In [174]:
class Deck:
    def __init__(self):
        self.cards = [Card(value,suit)\
                      for suit in ["♠","♥","♦","♣"]\
                      for value in ["A",2,3,4,5,6,7,8,9,10,"J","Q","K"]]
    
    def shuffle(self):
        random.shuffle(self.cards)
    
    def draw(self):
        if self.cards:
            return self.cards.pop(0)
        else:
            return "No more cards"
        
    def remove(self, card):
        if card in self.cards:
            self.cards.remove(card)
            
    def __iter__(self):
        return iter(self.cards)
    
    def __len__(self):
        return len(self.cards)

In [118]:
baraja = Deck()

In [119]:
baraja.cards[:3]

[ A♠,  2♠,  3♠]

In [120]:
baraja.shuffle()

In [121]:
baraja.cards[:3]

[ A♠,  5♥,  4♠]

## Player
¡Tenemos ya una baraja de cartas! ¿Pero quien va a jugar? 
Pues vamos hacer un objeto Player para controlar lo que hacen los diferentes jugadores, que cartas tienen y si quieren una carta nueva a cada ronda o no.

Seguiremos usando los dunder methods que hemos visto antes para la representación de los objetos.

In [137]:
class Player:
    def __init__(self, name="Player"):
        self.name = name
        self.hand = []
    
    def take_card(self,card):
        self.hand.append(card)
        
    def hit(self):
        hit = input(f"{self.name}, do you want another card? [y|n] ")
        if hit == "y":
            return True
        elif hit == "n":
            return False
        else:
            print("Wrong input. Try y or n")
            return self.hit()

In [138]:
p = Player("Pepe")
baraja = Deck()

In [139]:
baraja.shuffle()

In [145]:
if p.hit():
    p.take_card(baraja.draw())

Pepe, do you want another card? [y|n] vfsdgsfgsg
Wrong input. Try y or n
Pepe, do you want another card? [y|n] fsfgsfg
Wrong input. Try y or n
Pepe, do you want another card? [y|n] sgsf
Wrong input. Try y or n
Pepe, do you want another card? [y|n] y


In [146]:
p.hand

[ 6♦,  7♠]

In [155]:
p_1,p_2 = Player("Pepe"),Player("Paco")
deck = Deck()
deck.shuffle()
for _ in range(2):
    p_1.take_card(deck.draw())
    p_2.take_card(deck.draw())

In [156]:
p_1.hand

[ K♦,  A♠]

In [157]:
p_2.hand

[ 6♥,  2♠]

## CPU

Nosotros vamos a jugar, ¿pero contra quien? Vamos a crear un tipo de Player especial llamado `CPU`. Para eso, usaremos de la herencia para aprovechar al maximo todo lo que ya hemos hecho en la clase anterior. La diferencia es que los `CPU` necesitaran un mecanismo para que jueguen solos. Construiremos una inteligencia artificial basada en la probabilidad y el conteo de cartas. 

No os preocupéis, aseguraremosnos de que la CPU no haga trampas y no sea capaz de ver nuestra carta oculta.

In [170]:
class CPU(Player):
    def __init__(self,name="CPU"):
        super().__init__(name)
        self.card_counting = Deck()
    
    def count_card(self, game):
        for card in self.hand:
            self.card_counting.remove(card)
        for player in game.players:
            for card in player.hand[1:]:
                self.card_counting.remove(card)
                
    def prob(self):
        target = 21 - sum(self.hand)
        self.probabilities = {
            21 : len([card for card in self.card_counting if card == target])\
                 /(len(self.card_counting)-1),
            "under":len([card for card in self.card_counting if card < target])\
                 /(len(self.card_counting)-1),
            "over":len([card for card in self.card_counting if card > target])\
                 /(len(self.card_counting)-1)
        }
        
    def hit(self):
        print(f"{self.name}, do you want another card? [y|n]")
        self.prob()
        pros = self.probabilities[21]+self.probabilities["under"]
        cons = self.probabilities["over"]
        if pros > cons:
            print("Yes")
            return True
        else:
            print("No")
            return False

# Game

Parece que ya tenemos todos los elementos necesarios para echar un partido. Pero en el espíritu de la Programación Orientada a Objetos, el partido también será un tipo de objeto que utilizará todos los otros. 😎

In [180]:
class Game:
    def __init__(self, list_players):
        self.players = list_players
        self.deck = Deck()
        self.deck.shuffle()
        self.deal()
        
        
    def deal(self):
        for _ in range(2):
            for player in self.players:
                player.take_card(self.deck.draw())
                
    def round_p(self):
        for player in self.players:
            print(player.hand)
            if isinstance(player,CPU):
                player.count_card(self)
                if player.hit():
                    player.take_card(self.deck.draw())
            else:
                if player.hit():
                    player.take_card(self.deck.draw())

## Play!

Ya está todo listo y podemos jugar.

Como preparamos de antemano todos los objetos para que se comporten como esperamos, el script en si que pone el partido en marcha será muy sencillo.

In [181]:
players = [Player("Pepe"), CPU()]
game = Game(players)

In [182]:
[p.hand for p in players]

[[ Q♣,  2♦], [ 8♠,  4♣]]

In [183]:
game.round_p()

[ Q♣,  2♦]
Pepe, do you want another card? [y|n] y
[ 8♠,  4♣]
CPU, do you want another card? [y|n]
Yes


In [184]:
[p.hand for p in game.players]

[[ Q♣,  2♦,  2♣], [ 8♠,  4♣,  6♥]]

In [185]:
game.round_p()

[ Q♣,  2♦,  2♣]
Pepe, do you want another card? [y|n] y
[ 8♠,  4♣,  6♥]
CPU, do you want another card? [y|n]
No


In [186]:
[p.hand for p in game.players]

[[ Q♣,  2♦,  2♣,  4♦], [ 8♠,  4♣,  6♥]]